In [70]:
import pandas as pd
import nltk

In [63]:
import cx_Oracle

def hero_to_df(Userid, Password, Host, Port, SID, query):

    dsn_tns = cx_Oracle.makedsn(host=Host, port=Port, sid=sid_name)
    conn = cx_Oracle.connect(user=Userid, password=Password, dsn=dsn_tns)

    df = pd.read_sql_query(query, conn)
    df.ABSTRACT = df.ABSTRACT.astype(str)
    df.TITLE = df.TITLE.astype(str)
    
    conn.close()
    
    return(df)

## HEROPRD
Userid = "rrice"
Password = "Temp02122020" 
Host = "herodb.rtpnc.epa.gov"
Port = "1521"
sid_name = "heroprd"

def defaultquery(query):
    return hero_to_df(Userid, Password, Host, Port, sid_name, query)

query = '''select reference_id, year, title, abstract
    from tbl_reference 
    where sdelete = 'No' 
    and reference_id in
    (select reference_id from tbl_reference_usage where usage_id = 2755 and isdeleted = 0)'''

df = defaultquery(query)
df.head()

,REFERENCE_ID,YEAR,TITLE,ABSTRACT
0,57688,1987,The relationship between iron status and lead ...,The absorption of lead from loops of small int...
1,57722,1988,Effect of lead on macrophage function,Lead (Pb) has been shown to alter various para...
2,57748,1988,"Blood lead, intelligence, reading attainment, ...",None
3,57781,1988,Rods are selectively altered by lead: I. Elect...,None
4,57782,1988,Rods are selectively altered by lead: II. Ultr...,None


In [82]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def nltk_process(words):

    tokens = word_tokenize(words)
    
    stemmed_words = []
    for word in tokens:
        stemmed_words.append(ps.stem(word))
    
    stopword_filtered = [w for w in stemmed_words if w not in stop_words]
    
    #remove digits and punctuation   
    pattern = re.compile(r'[a-zA-Z]+')
    cleaned = [x for x in stopword_filtered if pattern.search(x)]
    
    return cleaned
    
nltkdf = pd.DataFrame()
nltkdf['Text'] = df['TITLE']+df['ABSTRACT'].dropna()  ###########changed to abstract
nltkdf['Text_Processed'] = nltkdf['Text'].apply(lambda x: nltk_process(x))

In [84]:
nltkdf.join(df).iloc[:,:].to_pickle('nltk_stemmed.pkl')

In [83]:
nltkdf['Text_Processed']

0       [relationship, iron, statu, lead, absorpt, rat...
1       [effect, lead, macrophag, functionlead, Pb, ha...
2       [blood, lead, intellig, read, attain, behaviou...
3       [rod, select, alter, lead, I., electrophysiolo...
4       [rod, select, alter, lead, II, ultrastructur, ...
                              ...                        
3183    [evalu, icrp, lead, biokinet, model, empir, co...
3184    [reactiv, pbso4, caco3, particl, relev, modif,...
3185    [integr, scienc, assess, particul, matter, fir...
3186    [lead, isotop, ratio, tree, bark, pocket, An, ...
3187    [metal, sourc, exposur, home, young, children,...
Name: Text_Processed, Length: 3188, dtype: object